# Vessels API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
And put your API key in a `SIGNAL_OCEAN_API_KEY` environment variable.

In [ ]:
from signal_ocean.vessels import VesselsAPI
import pandas as pd
import seaborn as sns
from datetime import datetime

## Call the vessels API
The Vessels API retrieves vessel information.

In [ ]:
api = VesselsAPI()

#### Get vessel classes

In [ ]:
vessel_classes = api.get_vessel_classes()

In [ ]:
aframax_vesel_class = next(vc for vc in vessel_classes if vc.name=='Aframax')
aframax_vesel_class

In [ ]:
pd.DataFrame([vc.__dict__ for vc in vessel_classes]).tail(5)

#### Get vessel types

In [ ]:
vessel_types = api.get_vessel_types()

In [ ]:
tanker_vessel_type = next(vt for vt in vessel_types if vt.name=='Dry')
tanker_vessel_type

In [ ]:
pd.DataFrame([vc.__dict__ for vc in vessel_types]).tail(3)

#### Get details for a specific vessel

In [ ]:
imo = 9436006
v = api.get_vessel(imo)
print(f'{v.imo}: {v.vessel_name} ({v.vessel_class} / {v.commercial_operator})')

#### Get details for all vessels

In [ ]:
vessels = api.get_vessels()

In [ ]:
len(vessels)

In [ ]:
df = pd.DataFrame([x.__dict__ for x in vessels])

In [ ]:
df.columns

In [ ]:
df[['imo', 'vessel_name', 'vessel_class']].sample(10)

#### Find fleet size per vessel class

In [ ]:
df[pd.isnull(df['scrapped_date'])]['vessel_class'].value_counts().to_frame('vessel_count').head(10)

#### Find the commercial operators that currently operate the largest Aframax fleets

In [ ]:
data = df[(df['vessel_class']=='Aframax')&(pd.isnull(df['scrapped_date']))]
data['commercial_operator'].value_counts().head(10)

#### Visualize fleet age by vessel class for VLCC, Suezmax and Aframax Tankers

In [ ]:
df['vessel_age'] = df.apply(lambda r: datetime.now().year - r['year_built'], axis=1)
data = df[(pd.isnull(df['scrapped_date']))&(df['vessel_class'].isin(['VLCC', 'Suezmax', 'Aframax']))]
sns.kdeplot(data=data, x='vessel_age', hue='vessel_class', multiple="stack");

#### Visualize pairwise relationships for deadweight, length_overall and breadth_extreme for VLCC, Suezmax and Aframax Tankers

In [ ]:
data = df[(pd.isnull(df['scrapped_date']))&(df['vessel_class'].isin(['VLCC', 'Suezmax', 'Aframax']))]
sns.pairplot(data, kind='kde', hue='vessel_class', vars=['deadweight', 'length_overall', 'breadth_extreme']);

#### Get all vessels the name of which contains the term signal

In [ ]:
vessels = api.get_vessels('signal')

In [ ]:
len(vessels)

In [ ]:
df = pd.DataFrame([x.__dict__ for x in vessels])

In [ ]:
df.head(3)